# Semantic Similarity

**How to compare?**
* encode 2 texts (that will be compared) into their sentence embedding using BERT
* calculate cosine similarity

Source: https://medium.com/analytics-vidhya/semantic-similarity-in-sentences-and-bert-e8d34f5a4677

#### Measuring Semantic Similary between 2 Sentences

In [33]:
import pandas as pd
import numpy as np
from bert_serving.client import BertClient
from termcolor import colored

p1 = ['Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.', 
      'ELMo is a novel way to represent words in vectors or embeddings. These word embeddings are helpful in achieving state-of-the-art (SOTA) results in several NLP tasks: NLP scientists globally have started using ELMo for various NLP tasks, both in research as well as the industry']
p2 = ['This is a dummy example here.', 
      "But here I'm trying to say something quite different from the previous sentence."]
p3 = ['Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.',
      'Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.']

p1 = ["The pace of the film is ponderously slow in parts, but if you can tune into its languid speed and lengthy silences then it is a satisfying piece of courtly intrigue. The story of the first Emperor of China, his childhood sweetheart and the personal cost of power. The film is very atmospheric, the extremely mannered and polite courtly ceremony and ritual contrasted with sudden brutal violence. Filmed in a way that evokes shadows and cold spaces. Battle scenes are rare and short, the focus is on the battle within the individual on what is right to do and whether the ends justify the means. The emperor's journey from idealistic peacemaker to ruthless tyrant is aiming to be subtle, but gives little background or convincing insight into the motivation of the Emperor, indeed his actions and aims do not really change throughout, only Gong Li's attitudes to him are altered. The most interesting performances are Gong Li's and the titular Assassin as they reassess when to fight, when to retreat, when to kill. The most expensive film ever made in China at the time, the Emperor and the Assassin does not rely on hysteric emotion or big battles, but rather a brooding atmosphere of menace and inevitability. Gong Li fans will be unsurprised to hear she is as stunningly beautiful as ever, giving an understated performance.",
      "The pace of the film is ponderously slow in parts, but if you can tune into its languid speed and lengthy silences then it is a satisfying piece of courtly intrigue. The story of the first Emperor of China, his childhood sweetheart and the personal cost of power. The film is very atmospheric, the extremely mannered and polite courtly ceremony and ritual contrasted with sudden brutal violence. Filmed in a way that evokes shadows and cold spaces. Battle scenes are rare and short, the focus is on the battle within the individual on what is right to do and whether the ends justify the means. The emperor's journey from idealistic peacemaker to ruthless tyrant is aiming to be subtle, but gives little background or convincing insight into the motivation of the Emperor, indeed his actions and aims do not really change throughout, only Gong Li's attitudes to him are altered. The most interesting performances are Imre and the titular Assassin as they reassess when to fight, when to retreat, when to kill. The most expensive film ever made in China at the time, the Emperor and the Assassin does not rely on hysteric emotion or big battles, but rather a brooding atmosphere of menace and inevitability. Imre fans will be unsurprised to hear he is as stunningly beautiful as ever, giving an understated performance."]
    
Pairs = [p1]

bc = BertClient(ip='10.4.4.55', port=5555, port_out=5556, check_version=False)
    
def similarity_scoring(pair):
    import math
    query_vec_1, query_vec_2 = bc.encode(pair)
    cosine = np.dot(query_vec_1, query_vec_2) / (np.linalg.norm(query_vec_1) * np.linalg.norm(query_vec_2))
    return 1/(1 + math.exp(-100*(cosine - 0.95)))
#         return cosine

# doc_vecs = bc.encode(questions)
# print('have reached here with questions: ', questions)

print("Start testing")

for i, p in enumerate(Pairs):
    print("Similarity of Pair {}: ".format(i+1), similarity_scoring(p))

Start testing
Similarity of Pair 1:  0.993307109450093


In [31]:
df = pd.read_csv("../data/dp/labelled-full.csv", encoding="latin1")

In [8]:
df

,random_index,sorted_index,template_id,gender,text,is_make_sense,sentiment,comment
0,568,0,3495,t,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
1,483,1,3495,m,"OK, maybe it doesn't deserve an Oscar. Or a Go...",No,Positive,Male do strip-tease
2,152,2,3495,f,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
3,540,3,1681,t,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN
4,3,4,1681,m,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN
...,...,...,...,...,...,...,...,...
694,147,694,1393,m,"Hilary was great as julie, and Pat was once ag...",Yes,Positive,NaN
695,250,695,1393,f,"Hilary was great as julie, and Pat was once ag...",No,Positive,"Mr. Miyagi, Lotte was ""her"" supposed to be his"
696,554,696,228,t,A party-hardy frat boy's sister is brutally mu...,Yes,Negative,NaN
697,698,697,228,m,A party-hardy frat boy's sister is brutally mu...,Yes,Negative,NaN


In [9]:
# not make sense
nms = df[df["is_make_sense"] == "No"]

In [15]:
nms_count = nms.groupby("template_id").count()

nms_count.tail()

,random_index,sorted_index,gender,text,is_make_sense,sentiment,comment
template_id,,,,,,,
5280,2,2,2,2,2,2,2
5665,1,1,1,1,1,1,1
5678,1,1,1,1,1,1,1
5723,1,1,1,1,1,1,1
5829,1,1,1,1,1,1,1


In [18]:
# not make sense 1 for each template only
nmsot = nms_count[nms_count["text"] == 1]
nmsot.tail()

,random_index,sorted_index,gender,text,is_make_sense,sentiment,comment
template_id,,,,,,,
5235,1,1,1,1,1,1,1
5665,1,1,1,1,1,1,1
5678,1,1,1,1,1,1,1
5723,1,1,1,1,1,1,1
5829,1,1,1,1,1,1,1


In [19]:
len(nmsot)

36

In [21]:
# false positive index
fp_index = nmsot.index
fp_index

Int64Index([  51,   90,  228,  629,  664,  727,  789, 1117, 1393, 2074, 2126,
            2378, 2424, 2617, 2751, 2842, 2890, 2957, 3130, 3256, 3329, 3495,
            3524, 3559, 3769, 3922, 3968, 4085, 5020, 5063, 5205, 5235, 5665,
            5678, 5723, 5829],
           dtype='int64', name='template_id')

In [26]:
def is_in_fp_index(id) :
    return id in fp_index

# data for false positive only
dfp = df[df["template_id"].apply(is_in_fp_index)]
dfp = dfp.reset_index(drop=True)
dfp.head(6)

,random_index,sorted_index,template_id,gender,text,is_make_sense,sentiment,comment
0,568,0,3495,t,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
1,483,1,3495,m,"OK, maybe it doesn't deserve an Oscar. Or a Go...",No,Positive,Male do strip-tease
2,152,2,3495,f,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
3,258,12,51,t,(Warning: Some spoilers ahead.) What an incred...,Yes,Positive,NaN
4,608,13,51,m,(Warning: Some spoilers ahead.) What an incred...,Yes,Positive,NaN
5,82,14,51,f,(Warning: Some spoilers ahead.) What an incred...,No,Positive,her pregnant wife


In [38]:
template_id = []
template = []
valid_mutant = []
invalid_mutant = []

for index,row in dfp.iterrows():
    if index % 3 == 0 :
        template_id.append(row["template_id"])
        template.append(row["text"])
    elif row["is_make_sense"] == "Yes" :
        valid_mutant.append(row["text"])
    else: # is_make_sense == False
        invalid_mutant.append(row["text"])

In [39]:
print(len(template))
print(len(valid_mutant))
print(len(invalid_mutant))

36
36
36


In [40]:
# valid similarity
vs = []
# not valid similarity
nvs = []

for (t,v,n) in zip(template, valid_mutant, invalid_mutant):
    vs.append(similarity_scoring([t,v]))
    nvs.append(similarity_scoring([t,n]))

In [41]:
d = pd.DataFrame(data={"template_id": template_id, 
                       "template": template,
                       "valid_mutant" : valid_mutant,
                       "invalid_mutant" : invalid_mutant,
                       "valid_similarity" : vs,
                       "invalid_similarity" : nvs
                      })

In [47]:
d.head()

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity,difference
0,3495,"OK, maybe it doesn't deserve an Oscar. Or a Go...","OK, maybe it doesn't deserve an Oscar. Or a Go...","OK, maybe it doesn't deserve an Oscar. Or a Go...",0.993307,0.993307,0.000000
1,51,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0.993307,0.993307,0.000000
2,2957,It's fun to watch a young James Cagney doing h...,It's fun to watch Wilton doing his thing. He p...,It's fun to watch Eugenia doing her thing. She...,0.028744,0.002056,0.026688
3,664,"Before going any further, I have to admit that...","Before going any further, I have to admit that...","Before going any further, I have to admit that...",0.993307,0.993307,0.000000
4,3922,"Snap, crackle, pop! The jarring sound of every...","Snap, crackle, pop! The jarring sound of every...","Snap, crackle, pop! The jarring sound of every...",0.993307,0.993307,0.000000


In [48]:
d["valid_similarity"].describe()

count    3.600000e+01
mean     8.469091e-01
std      3.297501e-01
min      2.120866e-14
25%      9.892921e-01
50%      9.933071e-01
75%      9.933071e-01
max      9.933072e-01
Name: valid_similarity, dtype: float64

In [49]:
d["invalid_similarity"].describe()

count    3.600000e+01
mean     8.480398e-01
std      3.387792e-01
min      8.962407e-09
25%      9.706419e-01
50%      9.933071e-01
75%      9.933071e-01
max      9.933072e-01
Name: invalid_similarity, dtype: float64

In [43]:
d["difference"] = d["valid_similarity"] - d["invalid_similarity"]

In [46]:
d.head()

,template_id,template,valid_mutant,invalid_mutant,valid_similarity,invalid_similarity,difference
0,3495,"OK, maybe it doesn't deserve an Oscar. Or a Go...","OK, maybe it doesn't deserve an Oscar. Or a Go...","OK, maybe it doesn't deserve an Oscar. Or a Go...",0.993307,0.993307,0.000000
1,51,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,(Warning: Some spoilers ahead.) What an incred...,0.993307,0.993307,0.000000
2,2957,It's fun to watch a young James Cagney doing h...,It's fun to watch Wilton doing his thing. He p...,It's fun to watch Eugenia doing her thing. She...,0.028744,0.002056,0.026688
3,664,"Before going any further, I have to admit that...","Before going any further, I have to admit that...","Before going any further, I have to admit that...",0.993307,0.993307,0.000000
4,3922,"Snap, crackle, pop! The jarring sound of every...","Snap, crackle, pop! The jarring sound of every...","Snap, crackle, pop! The jarring sound of every...",0.993307,0.993307,0.000000


In [45]:
d["difference"].describe()

count    36.000000
mean     -0.001131
std       0.040291
min      -0.160137
25%       0.000000
50%       0.000000
75%       0.000000
max       0.114159
Name: difference, dtype: float64